In [280]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
import keras
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import json
import seaborn as sns
%matplotlib inline

- Clean notebook to be only new 
- Be able to merge train features and train targets
- same with test
- be able to then split and run program
- test for overfit
- create additional features (business/extra features from swing/ extra features from bugs)
- run reviews for NLP

In [281]:
list_of_file_names = glob.glob("./zip_2/*.csv")

In [282]:
read_data_list_2 = []
fail_list = []
for i, data in enumerate(list_of_file_names):
    try:
        data = pd.read_csv(data)
        #data = unicode(data, errors='replace')
        read_data_list_2.append(data)
    except UnicodeDecodeError:
        fail_list.append(i)
#read_data_list
#fail_list
list_of_file_names_org = list_of_file_names

In [283]:
fail_list

[3]

In [284]:
for x in fail_list:
    del list_of_file_names[x]

In [285]:
list_of_file_names[3]

'./zip_2/Zip_Listings_PriceCut_SeasAdj_AllHomes.csv'

In [286]:
# build loop for data we can work with. Only dates and zip
def build_useful_df(data_list):
    useful_df = []
    for dataset in range(len(data_list)):
        data = read_data_list_2[dataset]
        region_name = data['RegionName']
        region_name = region_name.astype(str)
        new_df = data.select_dtypes(include=['float64'])
        new_df.insert(loc=0, column='RegionName', value=region_name) 
        new_df = new_df.sort_values('RegionName',ascending=False)
        new_df = new_df.set_index('RegionName')
        useful_df.append(new_df)
    return useful_df

In [287]:
useful_df = build_useful_df(read_data_list_2)


In [288]:
sixteen_list = []
for i, file in enumerate(useful_df):
    if len(file) >= 10000:
        sixteen_list.append(i)

In [289]:
len(sixteen_list)

30

In [290]:
len(useful_df)
for num in range(len(useful_df)):
    useful_df[num].dropna(inplace=True)

In [291]:
sixteen_good = [3, 14, 26, 53, 54, 55, 58, 66, 69, 73, 82]

In [292]:
target_data = useful_df[23]

In [293]:
final_data_list = []
for i, data in enumerate(useful_df):
    if i in sixteen_good:
        final_data_list.append(data)

In [294]:
# Docs that don't need feature engineering. They are already aggregates
pure_features = [0, 71, 75, 77, 79, 81]
useful_df[0].head()

,PctPriceCut,DaysOnMarket,BuyerSellerIndex,BuyerSellerIndexMetro
RegionName,,,,
99654,11.142857,82.0,9.230769,7.260083
99645,7.589286,74.0,0.769231,7.260083
99577,8.108108,79.5,4.615385,7.260083
99567,9.836066,76.0,2.307692,7.260083
99518,14.925373,85.0,10.000000,7.260083


In [295]:
def make_features(df, past_time_string, now_string):
    #df.dropna(inplace=True)
    features = pd.DataFrame()
    #features['RegionName'] = df['RegionName']
    mean = df.loc[:, past_time_string : now_string].mean(axis=1)
    features['mean'] = mean
    std = df.loc[:, past_time_string : now_string].std(axis=1)
    features['std'] = std
    mn = df.loc[:, past_time_string : now_string].min(axis=1)
    features['min'] = mn
    mx = df.loc[:, past_time_string : now_string].max(axis=1)
    features['max'] = mx
    features['swing'] = mx - mn
    change = df[now_string] - df[past_time_string]
    features['change'] = change
    mean_swing = features['swing'].mean()
    features['swing_pos'] = np.where(features['swing']>mean_swing, 1, 0)
    big_swing = features['swing'].std() + mean_swing
    features['swing_big'] = np.where(features['swing']>big_swing, 1, 0)
    features['swing_neg'] = np.where(features['swing']<mean_swing, 1, 0)
    swing_big_loss = mean_swing - features['swing'].std() 
    features['swing_loss_big'] = np.where(features['swing']<swing_big_loss, 1, 0)
    features.dropna(inplace=True)
    
    features = features.set_index(df.index)
    #print(features.iloc[1, :])
    return features
    
# List of data frames only on one now_time
def make_modeling_data(df_list, df_for_target, now_string):
    df_one = pd.DataFrame()
    target = pd.DataFrame()
    now_time = pd.to_datetime(now_string)
    now_value = df_for_target[now_string]
    future_time = now_time + timedelta(days=6*31)
    future_time_string= future_time.strftime("%Y-%m")
    future_value = df_for_target[future_time_string]
    target['target'] = future_value/now_value
    
    
    past_time = now_time - timedelta(days=6*31)
    past_time_string= past_time.strftime("%Y-%m")
    df_to_use_for_features_org= df_list[0].loc[:, :now_string]
    features_org = make_features(df_to_use_for_features_org, past_time_string, now_string)
    df_one = pd.merge(df_one, features_org, left_index=True, right_index=True, how = 'right')
    for i, df in enumerate(df_list[1:]):
        ind = str(i)
        columns = df.columns
        if '2011-01' in columns and '2012-01' in columns and '2013-01' in columns and '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
            df_to_use_for_features= df.loc[:, :now_string]
            features = make_features(df_to_use_for_features, past_time_string, now_string)
            df_one = pd.merge(df_one, features, right_index=True, left_index=True, how='inner')
    target = target.loc[df_one.index]    
    #now_time = pd.to_datetime(now_string)
    #now_value = df_for_target[now_string]
    #future_time = now_time + timedelta(days=6*31)
    #future_time_string= future_time.strftime("%Y-%m")
    #future_value = df_for_target[future_time_string]
            #target = future_value/now_value

    
    return df_one, target




test_features, test_targets = make_modeling_data(final_data_list, target_data, "2017-06")
train_features, train_targets = make_modeling_data(final_data_list, target_data, "2016-12")
extra_train_features_1, extra_train_targets_1 = make_modeling_data(final_data_list, target_data, '2016-06')
train_features = train_features.append(extra_train_features_1)
train_targets = train_targets.append(extra_train_targets_1)






#df = useful_df[num]
#columns = df.columns
#"2012", "2012", "2013",
date_counter = 0
for year in ["2012", "2013", "2014", "2015"]:
    for month in ["06","12"]:
            new_time = year+"-"+month
            date_counter += 1
            #if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
                    #extra_train_features, extra_train_targets = make_modeling_data(df, new_time)
                    #train_features = train_features.append(extra_train_features)
                    #train_targets = train_targets.append(extra_train_targets)
            extra_train_features, extra_train_targets = make_modeling_data(final_data_list, target_data, new_time)
            extra_train_features.dropna(inplace=True)
            train_features = train_features.append(extra_train_features)
            train_targets = train_targets.append(extra_train_targets)

In [296]:
train_features.shape

(10520, 110)

In [297]:
train_features.dropna(inplace=True)
train_targets.dropna(inplace=True)

In [298]:
test_features.to_csv('test_features_to_merge_final.csv')

In [299]:
train_features.head()

,mean_x,std_x,min_x,max_x,swing_x,change_x,swing_pos_x,swing_big_x,swing_neg_x,swing_loss_big_x,...,mean,std,min,max,swing,change,swing_pos,swing_big,swing_neg,swing_loss_big
RegionName,,,,,,,,,,,,,,,,,,,,,
99223,14.939552,2.850747,10.612033,19.305586,8.693553,3.061711,1,0,0,0,...,0.86650,0.005425,0.856,0.874,0.018,0.018,0,0,1,0
99218,18.229328,6.169784,11.025478,27.286549,16.261071,5.440417,1,1,0,0,...,0.82325,0.006497,0.812,0.834,0.022,0.022,0,0,1,0
99217,14.059069,3.273154,9.914238,17.699958,7.785721,0.363912,0,0,1,0,...,0.87525,0.003196,0.872,0.882,0.010,0.010,0,0,1,0
99212,13.873385,3.268283,8.633957,18.783999,10.150042,6.561347,1,0,0,0,...,0.91375,0.002712,0.910,0.918,0.008,0.006,0,0,1,0
99208,14.519844,1.530183,12.713332,16.503253,3.789921,-0.064371,0,0,1,1,...,0.86000,0.004660,0.850,0.866,0.016,0.016,0,0,1,0


In [300]:
train_targets.shape

(10480, 1)

In [301]:
business_features = pd.read_csv('real_business_to_merge_with_zillow')

In [302]:
business_features.head()

,postal_code,is_open,latitude,longitude,review_count,stars,average_review_count,business_count,zip_review_count,average_review_max,average_review_min,average_review_variance
0,2224,1.0,41.499208,-81.536689,7.000000,2.500000,7.000000,1.0,7.0,7.0,7.0,0.0
1,5440,1.0,44.964078,-73.285549,4.000000,4.250000,4.000000,2.0,8.0,5.0,3.0,2.0
2,5452,1.0,40.780821,-74.150722,49.000000,4.000000,49.000000,1.0,49.0,49.0,49.0,0.0
3,6618,1.0,51.083200,11.858200,8.000000,3.500000,8.000000,1.0,8.0,8.0,8.0,0.0
4,6632,1.0,51.213256,11.763245,5.888889,4.222222,5.888889,9.0,53.0,8.0,3.0,5.0


In [303]:
business_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 12 columns):
postal_code                959 non-null int64
is_open                    959 non-null float64
latitude                   959 non-null float64
longitude                  959 non-null float64
review_count               959 non-null float64
stars                      959 non-null float64
average_review_count       959 non-null float64
business_count             959 non-null float64
zip_review_count           959 non-null float64
average_review_max         959 non-null float64
average_review_min         959 non-null float64
average_review_variance    959 non-null float64
dtypes: float64(11), int64(1)
memory usage: 90.0 KB


In [304]:
business_features.shape

(959, 12)

In [305]:
business_features['postal_code'] = business_features['postal_code'].astype(str)

In [306]:
business_features['postal_code'].nunique()

959

In [307]:
business_features = business_features.set_index('postal_code')

In [308]:
business_features.iloc[:, 3:].head()

,review_count,stars,average_review_count,business_count,zip_review_count,average_review_max,average_review_min,average_review_variance
postal_code,,,,,,,,
2224,7.000000,2.500000,7.000000,1.0,7.0,7.0,7.0,0.0
5440,4.000000,4.250000,4.000000,2.0,8.0,5.0,3.0,2.0
5452,49.000000,4.000000,49.000000,1.0,49.0,49.0,49.0,0.0
6618,8.000000,3.500000,8.000000,1.0,8.0,8.0,8.0,0.0
6632,5.888889,4.222222,5.888889,9.0,53.0,8.0,3.0,5.0


In [309]:
business_features = business_features.iloc[:, 3:]

In [310]:
business_features.shape

(959, 8)

In [311]:
train_features.shape

(10520, 110)

In [312]:
train_features_business = pd.merge(train_features, business_features,  right_index=True, left_index=True, how='inner')

In [313]:
test_features_business = pd.merge(test_features, business_features,  right_index=True, left_index=True, how='inner')

In [314]:
train_features_business.shape

(550, 118)

In [315]:
# The way to extrapolate smaller df. Now must finish. Smaller must hae index like merge column
train_merge = pd.merge(train_features, train_targets, right_index=True, left_index=True, how='inner')

In [316]:
test_merge = pd.merge(test_features, test_targets, right_index=True, left_index=True, how='inner')

In [317]:
train_business_merge = pd.merge(train_features_business, train_targets, right_index=True, left_index=True, how='inner')

In [318]:
test_business_merge = pd.merge(test_features_business, test_targets, right_index=True, left_index=True, how='inner')

In [319]:
train_merge.dropna(inplace=True)

In [320]:
test_merge.dropna(inplace=True)

In [321]:
train_business_merge.shape

(5400, 119)

In [322]:
train_business_merge.head()

,mean_x,std_x,min_x,max_x,swing_x,change_x,swing_pos_x,swing_big_x,swing_neg_x,swing_loss_big_x,...,swing_loss_big,review_count,stars,average_review_count,business_count,zip_review_count,average_review_max,average_review_min,average_review_variance,target
15102,14.017885,1.704769,11.28749,16.568575,5.281086,2.21851,0,0,1,0,...,0,10.927711,3.692771,10.927711,166.0,1814.0,130.0,3.0,127.0,1.038092
15102,14.017885,1.704769,11.28749,16.568575,5.281086,2.21851,0,0,1,0,...,0,10.927711,3.692771,10.927711,166.0,1814.0,130.0,3.0,127.0,0.986697
15102,14.017885,1.704769,11.28749,16.568575,5.281086,2.21851,0,0,1,0,...,0,10.927711,3.692771,10.927711,166.0,1814.0,130.0,3.0,127.0,1.006789
15102,14.017885,1.704769,11.28749,16.568575,5.281086,2.21851,0,0,1,0,...,0,10.927711,3.692771,10.927711,166.0,1814.0,130.0,3.0,127.0,1.036640
15102,14.017885,1.704769,11.28749,16.568575,5.281086,2.21851,0,0,1,0,...,0,10.927711,3.692771,10.927711,166.0,1814.0,130.0,3.0,127.0,1.014253


In [323]:
X_train = train_merge.iloc[:, :-1].values
y_train = train_merge.iloc[:, -1].values
X_test = test_merge.iloc[:, :-1].values
y_test = test_merge.iloc[:, -1].values

In [324]:
X_train_business = train_business_merge.iloc[:, :-1].values
y_train_business = train_business_merge.iloc[:, -1].values
X_test_business = test_business_merge.iloc[:, :-1].values
y_test_business = test_business_merge.iloc[:, -1].values

In [325]:
X_train_business.shape

(5400, 118)

In [326]:
def norm_per_column(X):
    new_array = []
    shape = X.shape
    columns = shape[1]
    for num in range(columns):
        mx = X[:, num].max()
        if mx == 0:
            new_array.append(X[:, num])
        else:
            X[:, num] = X[:, num] / mx
            new_array.append(X[:, num])
    new_array = np.array(new_array)
    new_array = np.transpose(new_array)
    return new_array

In [327]:
X_train = norm_per_column(X_train)
X_test = norm_per_column(X_test)

In [328]:
X_train_business = norm_per_column(X_train_business)
X_test_business = norm_per_column(X_test_business)

In [329]:
X_train_business.shape

(5400, 118)

In [62]:
# Hold overfit model for now
model = Sequential()
model.add(Dense(3000, input_shape=(110,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [81]:
model = Sequential()
model.add(Dense(1000, input_shape=(110,), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [83]:
model.fit(x=X_train, y=y_train, 
          batch_size=5000, 
          epochs=40, 
          verbose=1, 
          validation_data=(X_test, y_test),
          callbacks=None)

Train on 126808 samples, validate on 1048 samples
Epoch 1/40
126808/126808 [==============================] - 31s 247us/step - loss: 0.0025 - val_loss: 0.0050
Epoch 2/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 3/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0052
Epoch 4/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0049
Epoch 5/40
126808/126808 [==============================] - 31s 245us/step - loss: 0.0025 - val_loss: 0.0042
Epoch 6/40
126808/126808 [==============================] - 31s 247us/step - loss: 0.0025 - val_loss: 0.0056
Epoch 7/40
126808/126808 [==============================] - 31s 246us/step - loss: 0.0025 - val_loss: 0.0073
Epoch 8/40
126808/126808 [==============================] - 31s 245us/step - loss: 0.0025 - val_loss: 0.0057
Epoch 9/40
126808/126808 [==============================] - 31s 244us/step - l

In [ ]:
#loss = .00001441 (overfit) = .0038

#std = .072

In [278]:
model = Sequential()
model.add(Dense(3000, input_shape=(118,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [279]:
model.fit(x=X_train_business, y=y_train_business, 
          batch_size=5000, 
          epochs=40, 
          verbose=1, 
          validation_data=(X_test_business, y_test_business),
          callbacks=None)

Train on 5400 samples, validate on 55 samples
Epoch 1/40
5400/5400 [==============================] - 7s 1ms/step - loss: 219268962.0741 - val_loss: nan
Epoch 2/40
5400/5400 [==============================] - 6s 1ms/step - loss: 353883981.0370 - val_loss: nan
Epoch 3/40
5400/5400 [==============================] - 6s 1ms/step - loss: 94313633.6667 - val_loss: nan
Epoch 4/40
5400/5400 [==============================] - 6s 1ms/step - loss: 62565846.3069 - val_loss: nan
Epoch 5/40
5400/5400 [==============================] - 7s 1ms/step - loss: 41941841.7685 - val_loss: nan
Epoch 6/40


KeyboardInterrupt: 